In [13]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (2).json


{'kaggle.json': b'{"username":"minzzzzi","key":"74a980fb52033cf0ead89c84120a4a9c"}'}

In [14]:
!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle

Found existing installation: kaggle 1.5.8
Uninstalling kaggle-1.5.8:
  Successfully uninstalled kaggle-1.5.8
Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (20.2.4)
Processing /root/.cache/pip/wheels/cf/aa/f0/ed1179bbcd729b29d0dfda59826fb3b55f0a4a0c3f713c1c82/kaggle-1.5.8-py3-none-any.whl


In [16]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [17]:
!kaggle competitions download -c 2020ai-project-18011797

  0% 0.00/7.45k [00:00<?, ?B/s]
100% 7.45k/7.45k [00:00<00:00, 6.63MB/s]


In [18]:
!unzip 2020ai-project-18011797.zip

Archive:  2020ai-project-18011797.zip
replace waterquality_submit.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: waterquality_submit.csv  
  inflating: waterquality_test.csv   
  inflating: waterquality_train.csv  


In [19]:
import numpy as np
import pandas as pd

In [26]:
train = pd.read_csv('waterquality_train.csv',header=None, skiprows=1)
test = pd.read_csv('waterquality_test.csv',header=None, skiprows=1)
submit = pd.read_csv('waterquality_submit.csv')

train[:4]

,0,1,2,3,4,5,6,7,8
0,1538,11.2,7.1,1.3,4.4,0.4,6.400,0.030,21
1,1498,11.1,7.0,1.7,6.4,2.6,3.931,0.030,26
2,1905,11.3,7.1,0.7,6.9,2.0,4.241,0.027,16
3,1492,14.2,7.1,0.7,7.3,2.0,4.912,0.033,17


In [27]:
train[0] = train[0]%10000/100
test[0] = test[0]%10000/100

In [35]:
x_train = train.loc[:,0:7]
y_train = train.loc[:,8]

x_test = test.loc[:,0:7]

In [36]:
x_train[:3]

,0,1,2,3,4,5,6,7
0,15.38,11.2,7.1,1.3,4.4,0.4,6.400,0.030
1,14.98,11.1,7.0,1.7,6.4,2.6,3.931,0.030
2,19.05,11.3,7.1,0.7,6.9,2.0,4.241,0.027


In [38]:
from sklearn.preprocessing import RobustScaler
sc = RobustScaler()

x_train_sc = sc.fit_transform(x_train)
x_test_sc = sc.fit_transform(x_test)

---
#KNN
---

In [39]:
from sklearn.metrics import mean_squared_error

In [48]:
from sklearn.neighbors import KNeighborsRegressor
KNN = KNeighborsRegressor(n_neighbors = 10, weights = 'distance',p = 2)

KNN.fit(x_train_sc,y_train)
train_pred = KNN.predict(x_train_sc)
print(mean_squared_error(y_train,train_pred))

0.0


In [49]:
test_pred = KNN.predict(x_test_sc)

for i in range(len(test_pred)):
  submit['Expected'][i] = test_pred[i]

submit.to_csv('submission.csv',header=True,index = None)
submit[:5]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Id,Expected
0,0,27.616963
1,1,30.057498
2,2,33.416269
3,3,24.353947
4,4,25.528608


In [50]:
!kaggle competitions submit -c 2020ai-project-18011797 -f submission.csv -m "KNN"

100% 1.05k/1.05k [00:09<00:00, 110B/s]
Successfully submitted to SejongAI.텀프로젝트.[단양하수처리장 수질 예측]

####(n_neighbors = 5, weights = 'distance',p = 2) / 12.91
####(n_neighbors = 3, weights = 'distance',p = 2) / 13.15
####(n_neighbors = 10, weights = 'distance',p = 2) / 13.47

---
#Logistic Regression 
---

In [60]:
from sklearn.linear_model import LogisticRegression
logit = LogisticRegression(C = 100.0,class_weight = 'balanced',random_state = 1, solver = 'sag',max_iter=2000)

logit.fit(x_train_sc,y_train)
train_pred = logit.predict(x_train_sc)
print(mean_squared_error(y_train,train_pred))

325.6970443349754


In [61]:
test_pred = logit.predict(x_test_sc)

for i in range(len(test_pred)):
  submit['Expected'][i] = test_pred[i]

submit.to_csv('submission.csv',header=True,index = None)
submit[:5]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Id,Expected
0,0,46.0
1,1,42.0
2,2,12.0
3,3,16.0
4,4,18.0


In [62]:
!kaggle competitions submit -c 2020ai-project-18011797 -f submission.csv -m "logit"

100% 402/402 [00:07<00:00, 56.0B/s]
Successfully submitted to SejongAI.텀프로젝트.[단양하수처리장 수질 예측]

####(C = 10.0,class_weight = 'balanced',random_state = 1, solver = 'sag',max_iter=1000) / 30.34

#### (C = 100.0,class_weight = 'balanced',random_state = 1, solver = 'sag',max_iter=2000) / 30.17

---
# Decision Tree
---

In [76]:
from sklearn.tree import DecisionTreeRegressor

regressor = DecisionTreeRegressor(random_state=1, max_depth = 5)
regressor.fit(x_train_sc, y_train)

train_pred = regressor.predict(x_train_sc)
print(mean_squared_error(y_train,train_pred))

124.65485433910838


In [77]:
test_pred = regressor.predict(x_test_sc)

for i in range(len(test_pred)):
  submit['Expected'][i] = test_pred[i]

submit.to_csv('submission.csv',header=True,index = None)
submit[:5]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Id,Expected
0,0,31.489796
1,1,21.458333
2,2,21.458333
3,3,21.458333
4,4,31.309524


In [78]:
!kaggle competitions submit -c 2020ai-project-18011797 -f submission.csv -m "DTC"

100% 951/951 [00:09<00:00, 103B/s]
Successfully submitted to SejongAI.텀프로젝트.[단양하수처리장 수질 예측]

####(random_state=1, max_depth = 7) / 19.64
####(random_state=1, max_depth = 3) / 20.28
####(random_state=1, max_depth = 5) / 19.64
####(criterion = 'mae',random_state=1, max_depth = 5) / 22.99